# SageMaker A/B Testing: A Practical Simulation

This notebook demonstrates how to implement and evaluate A/B testing strategies with Amazon SageMaker. A/B testing is a critical technique for data-driven decision making, allowing you to compare two variants of a machine learning model to determine which performs better in real-world scenarios.

Rather than deploying multiple model variants to production endpoints (which would incur significant AWS costs), this notebook uses simulation to illustrate the key concepts and analysis techniques of A/B testing. This simulation-based approach provides several advantages:

- **Cost-effective learning**: Explore A/B testing methodologies without incurring the expenses of running multiple SageMaker endpoints
- **Rapid experimentation**: Test different traffic allocation scenarios and analysis techniques instantly
- **Educational value**: Focus on understanding the metrics and decision-making process without debugging deployment issues

## Overview:
1. Set up simulated model variants with different performance characteristics
2. Distribute synthetic traffic between these variants
3. Collect and analyze conversion metrics over time
4. Visualize the results to identify the better-performing model
5. Make data-driven recommendations for full deployment

While simulation is used for this demonstration, the concepts, code patterns, and analysis techniques can be directly applied to real-world SageMaker deployments when you're ready to implement production A/B testing.

Let's begin by setting up our simulated A/B testing environment

In [1]:
import boto3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from datetime import datetime, timedelta

# Setup clients
sm = boto3.client('sagemaker')
sm_runtime = boto3.client('sagemaker-runtime')

# Define endpoint and variant configuration
endpoint_name = "xgb-churn-pred"
model_name = "xgb-churn-model"

# Define two variants for A/B testing
variant1 = {
    "VariantName": "ModelVariant1",  # Original model
    "ModelName": model_name,
    "InitialInstanceCount": 1,
    "InstanceType": "ml.m5.large",
    "InitialVariantWeight": 0.5  # 50% of traffic
}

variant2 = {
    "VariantName": "ModelVariant2",  # Improved model
    "ModelName": model_name,
    "InitialInstanceCount": 1,
    "InstanceType": "ml.m5.large",
    "InitialVariantWeight": 0.5  # 50% of traffic
}



In [ ]:
# For this example, we'll skip actually creating the endpoint
print(f"Demonstrating A/B testing for endpoint: {endpoint_name}")
print(f"Variant 1: {variant1['VariantName']} (Weight: {variant1['InitialVariantWeight']})")
print(f"Variant 2: {variant2['VariantName']} (Weight: {variant2['InitialVariantWeight']})")

# Simulate A/B test traffic and results
def simulate_ab_test(num_requests=100, days=5):
    """Simulate A/B test results over a period of days"""
    np.random.seed(42)  # For reproducibility
    
    # Create date range
    today = datetime.now().date()
    date_range = [today - timedelta(days=(days-i)) for i in range(1, days+1)]
    
    # Initialize results DataFrames
    traffic_data = []
    conversion_data = []
    
    # Model performance characteristics (simulate variant2 being slightly better)
    variant1_conversion_rate = 0.15
    variant2_conversion_rate = 0.18
    
    for date in date_range:
        # Randomly distribute traffic based on weights
        daily_requests = num_requests + np.random.randint(-10, 10)  # Add some noise
        variant1_traffic = int(daily_requests * variant1['InitialVariantWeight'])
        variant2_traffic = daily_requests - variant1_traffic
        
        # Calculate conversions based on simulated rates
        variant1_conversions = int(variant1_traffic * variant1_conversion_rate * (1 + np.random.normal(0, 0.1)))
        variant2_conversions = int(variant2_traffic * variant2_conversion_rate * (1 + np.random.normal(0, 0.1)))
        
        # Record data
        traffic_data.append({
            'date': date,
            'variant1_traffic': variant1_traffic,
            'variant2_traffic': variant2_traffic
        })
        
        conversion_data.append({
            'date': date,
            'variant1_conversions': variant1_conversions,
            'variant2_conversions': variant2_conversions
        })
    
    return pd.DataFrame(traffic_data), pd.DataFrame(conversion_data)



In [ ]:
# Run simulation
traffic_df, conversion_df = simulate_ab_test(num_requests=200, days=7)

# Display results
print("\nSimulated traffic distribution:")
print(traffic_df)
print("\nSimulated conversion results:")
print(conversion_df)

# Calculate metrics
results = pd.merge(traffic_df, conversion_df, on='date')
results['variant1_rate'] = results['variant1_conversions'] / results['variant1_traffic']
results['variant2_rate'] = results['variant2_conversions'] / results['variant2_traffic']
results['improvement'] = ((results['variant2_rate'] - results['variant1_rate']) / 
                         results['variant1_rate'] * 100)



In [ ]:
print("\nA/B Test Analysis:")
print(results[['date', 'variant1_rate', 'variant2_rate', 'improvement']])

# Calculate overall metrics
total_v1_traffic = results['variant1_traffic'].sum()
total_v2_traffic = results['variant2_traffic'].sum()
total_v1_conversions = results['variant1_conversions'].sum()
total_v2_conversions = results['variant2_conversions'].sum()
overall_v1_rate = total_v1_conversions / total_v1_traffic
overall_v2_rate = total_v2_conversions / total_v2_traffic
overall_improvement = ((overall_v2_rate - overall_v1_rate) / overall_v1_rate) * 100

print(f"\nOverall Results:")
print(f"Variant 1 Conversion Rate: {overall_v1_rate:.2%}")
print(f"Variant 2 Conversion Rate: {overall_v2_rate:.2%}")
print(f"Improvement (relative improvement of Variant 2 over Variant 1.): {overall_improvement:.2f}%")



In [ ]:
# Visualization
plt.figure(figsize=(12, 10))

# Plot 1: Traffic distribution
plt.subplot(2, 1, 1)
plt.title('A/B Test Traffic Distribution')
plt.plot(results['date'], results['variant1_traffic'], 'b-', label=f'Variant 1 ({variant1["VariantName"]})')
plt.plot(results['date'], results['variant2_traffic'], 'r-', label=f'Variant 2 ({variant2["VariantName"]})')
plt.xlabel('Date')
plt.ylabel('Number of Requests')
plt.legend()
plt.grid(True)

# Plot 2: Conversion rates
plt.subplot(2, 1, 2)
plt.title('A/B Test Conversion Rates')
plt.plot(results['date'], results['variant1_rate'], 'b-', label=f'Variant 1 ({variant1["VariantName"]})')
plt.plot(results['date'], results['variant2_rate'], 'r-', label=f'Variant 2 ({variant2["VariantName"]})')
plt.axhline(y=overall_v1_rate, color='b', linestyle='--', alpha=0.5)
plt.axhline(y=overall_v2_rate, color='r', linestyle='--', alpha=0.5)
plt.xlabel('Date')
plt.ylabel('Conversion Rate')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()



In [ ]:
# Recommendation based on A/B test results
if overall_improvement > 5:
    print("\nRecommendation: Deploy Variant 2 fully (100% traffic)")
elif overall_improvement > 0:
    print("\nRecommendation: Consider longer testing or deploy Variant 2")
else:
    print("\nRecommendation: Keep Variant 1 as the primary model")